In [1]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [2]:
import os
import torch
from huggingface_hub import HfFolder
from datasets import load_from_disk
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!unzip medical_clean_tokenized.zip

Archive:  medical_clean_tokenized.zip
   creating: tokenized_train_ds/
   creating: tokenized_train_ds/train/
  inflating: tokenized_train_ds/train/data-00002-of-00003.arrow  
  inflating: tokenized_train_ds/train/data-00000-of-00003.arrow  
  inflating: tokenized_train_ds/train/data-00001-of-00003.arrow  
  inflating: tokenized_train_ds/train/dataset_info.json  
  inflating: tokenized_train_ds/train/state.json  
   creating: tokenized_train_ds/eval/
  inflating: tokenized_train_ds/eval/dataset_info.json  
  inflating: tokenized_train_ds/eval/state.json  
  inflating: tokenized_train_ds/eval/data-00000-of-00001.arrow  


In [5]:
trainset = load_from_disk('tokenized_train_ds/train')
testset = load_from_disk('tokenized_train_ds/eval')

In [6]:
#model_id = "t5-small" # Hugging Face Model Id
model_id = "oliverli0414/t5-small_medical_finetuned" # Hugging Face Model Id
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [7]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [9]:
batch_size = 128
repository_id = "oliverli0414/t5-small_medical_finetuned"
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    generation_max_length=273,
    weight_decay=0.01,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=False,
    bf16=True,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    #load_best_model_at_end=True,
    # push to hub parameters
    report_to="tensorboard",
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    push_to_hub=True,
)

In [10]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-10-ff0810d3198b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [12]:
trainset[456]

{'__index_level_0__': 456,
 'input_ids': [30355,
  15,
  2379,
  12,
  1566,
  10,
  18027,
  25416,
  3,
  35,
  3,
  107,
  154,
  51,
  23193,
  120,
  7,
  15,
  10,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [20]:
print(sum(testset[456]['input_ids']))
print(sum(testset[456]['attention_mask']))
print(sum(testset[456]['labels']))
print(sum(trainset[456]['input_ids']))
print(sum(trainset[456]['attention_mask']))
print(sum(trainset[456]['labels']))

180680
38
104652
101479
20
50244


In [21]:
trainer.train()

Step,Training Loss
500,2.309400
1000,2.311400
1500,2.311300
2000,2.315200
2500,2.313000
3000,2.306400
3500,2.308700
4000,2.301600
4500,2.295100
5000,2.300600


Step,Training Loss
500,2.309400
1000,2.311400
1500,2.311300
2000,2.315200
2500,2.313000
3000,2.306400
3500,2.308700
4000,2.301600
4500,2.295100
5000,2.300600


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

